# Importing Dependencies

In [176]:
import pandas as pd

In [206]:
df=pd.read_csv('Deduplication Problem - Sample Dataset.csv')

In [178]:
df.head(8)

,ln,dob,gn,fn
0,SMITH JR,01-03-68,F,WILLIAM
1,ROTHMEYER JR,01-03-68,F,WILLIAM
2,ASBY JR,01-03-68,F,WILLIAM
3,SALTER JR,01-03-68,F,WILLIAM
4,SALTER JR,01-03-68,F,WILLIAM
5,BLAND JR,21-02-62,F,WILLIAM
6,BLAND JR,21-02-62,F,WILLIAM
7,BLAND JR,21-02-62,F,WILLIAM


In [179]:
df.describe()

,ln,dob,gn,fn
count,106,106,106,106
unique,31,23,2,27
top,MICHAELSON JR,07-10-37,M,HAROLD
freq,22,24,80,24


In [180]:
df.nunique()

ln     31
dob    23
gn      2
fn     27
dtype: int64

## Vectorizing the Text & Seperating Dates

In [181]:
from sklearn.feature_extraction.text import CountVectorizer


#from sklearn.feature_extraction.text import TfidfVectorizer


In [207]:
df['day']=pd.to_datetime(df['dob']).dt.day
df['month']=pd.to_datetime(df['dob']).dt.month
df['year']=pd.to_datetime(df['dob']).dt.year

In [208]:
df.drop('dob',axis=1,inplace=True)

In [209]:
gender_dummie=pd.get_dummies(df['gn'])

In [210]:
df=pd.concat([df,gender_dummie],axis=1)
df.drop('gn',axis=1,inplace=True)

In [211]:
from sklearn.cluster import KMeans
K=KMeans()

In [212]:
vectorizer = CountVectorizer()
Vect1=vectorizer.fit_transform(df['fn'])
vectorizer.get_feature_names()
FirstNameCol=pd.DataFrame(Vect1.toarray(),columns=vectorizer.get_feature_names())

vectorizer = CountVectorizer()
vectorizer=CountVectorizer()
Vect2=vectorizer.fit_transform(df['ln'])
SecondNameCol=pd.DataFrame(Vect2.toarray(),columns=vectorizer.get_feature_names())

In [213]:
Final=pd.concat([df,FirstNameCol,SecondNameCol],axis=1)

In [214]:
Final.drop(['ln','fn'],axis=1,inplace=True)


## Creating Manual Feature

In [215]:
from sklearn.metrics.pairwise import cosine_similarity


In [216]:
len_fn=[]
len_ln=[]

for i in range(len(df)):
    len_fn.append('fn'+str(i))
    len_ln.append('ln'+str(i))
    


In [217]:
fn_df=pd.DataFrame(3*cosine_similarity(Vect1[:], Vect1),columns=len_fn)
ln_df=pd.DataFrame(3*cosine_similarity(Vect2[:], Vect2),columns=len_ln)
Final_1=pd.concat([Final,fn_df,ln_df],axis=1)

In [218]:
2*cosine_similarity(Vect1[:], Vect1).shape

(106, 106, 106, 106)

documents = (
    "Dhruv Rawat",
    "Dhruv Raj Rawat",
    "Dhruv Raj Singh Rawat",
    "Dhruv asd"
    )
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
cosine_similarity(tfidf_matrix[:], tfidf_matrix).shape


# Evaluation

In [219]:
from sklearn.metrics import silhouette_score,silhouette_samples

In [220]:
for i in range(3,100):
    Kmeans_model=KMeans(n_clusters=i).fit(Final_1)
    labels=Kmeans_model.labels_
    print(i,silhouette_score(Final_1,labels))

3 0.34532688744979134
4 0.3668570761066961
5 0.39315888141036875
6 0.42313799433464966
7 0.44981732512735423
8 0.45253111209108615
9 0.49182858890939274
10 0.5143251607959993
11 0.5297457124353299
12 0.5260346534667377
13 0.5343005857066097
14 0.5601898589699134
15 0.5618798746186673
16 0.5862513378943534
17 0.5749148605985998
18 0.6146381990083721
19 0.6134987761773418
20 0.6428530792049946
21 0.6493984533819891
22 0.6346311627972301
23 0.6622064032191143
24 0.6624633436593649
25 0.6054526848795222
26 0.61175487941995
27 0.6244862401251438
28 0.6702824807764063
29 0.6646314328216474
30 0.6636289338968137
31 0.6972769395727368
32 0.6809386365349744
33 0.6751678829532399
34 0.6897330357247387
35 0.7027444252287398
36 0.7208930513378927
37 0.7192805610348382
38 0.7322919505388393
39 0.7282364032868134
40 0.7465027368921261
41 0.7517961656755097
42 0.7672215445217013
43 0.7645440331800258
44 0.7640010423613078
45 0.76520767568547
46 0.7565227361193424
47 0.7635725459499509
48 0.7418044217

In [229]:
Kmeans_model=KMeans(n_clusters=42).fit(Final_1)
labels=Kmeans_model.labels_

In [230]:
labels

array([ 1,  1,  1,  1,  1, 11, 11, 11, 38, 38, 21, 24, 21, 11, 16, 16,  2,
        2,  2,  7, 36,  7, 36, 18, 15, 18, 15, 39,  9,  9, 25, 14, 14, 28,
       28, 41, 41, 41, 10, 10, 33, 33, 30, 30, 22, 13, 13, 31,  0,  0, 26,
       26, 26,  8,  8,  8,  8,  8, 37, 37, 17, 35, 40, 40,  3,  3, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 27, 27, 27, 27, 32, 32,  3, 27, 27,  3,
        3,  5,  5, 34, 34, 19, 19, 19, 34, 23,  4, 23, 23, 23, 29, 12, 12,
       12,  6,  6,  6])

In [227]:
Score=pd.DataFrame(labels,columns=['Similarity'])

In [231]:
pd.concat([df,Score],axis=1).to_csv("clustered_2.csv",index=False)
